In [1]:
from pyspark import SparkContext

In [5]:
sc = SparkContext(master="local", appName="T&C")

In [6]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [7]:
rdd1.collect()

[1, 2, 3]

In [8]:
sc

<SparkContext master=local appName=T&C>

In [9]:
!ls files/

deporte.csv           deportistaError.csv   modelo_relacional.jpg
deportista.csv        evento.csv            paises.csv
deportista2.csv       juegos.csv            resultados.csv


In [10]:
PATH = 'files/'
equiposOlimpicosRDD = sc.textFile(PATH + 'paises.csv') \
    .map(lambda line : line.split(","))

In [11]:
equiposOlimpicosRDD.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [12]:
'''Counting different countries in the rdd'''
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [13]:
'''
    Getting the number of teams by country
    mapValues takes a function as argument
'''
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [14]:
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey().mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [15]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l: 'ARG' in l)
equiposArgentinos.take(18)

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [16]:
equiposOlimpicosRDD.count()

1185

In [17]:
'''
    Contar puede ser muy lento si tenemos muchos valores
    Podemos hacer un conteo aproximado limitando los milisegundos
'''

equiposOlimpicosRDD.countApprox(10)

1185

In [18]:
!ls files

deporte.csv           deportistaError.csv   modelo_relacional.jpg
deportista.csv        evento.csv            paises.csv
deportista2.csv       juegos.csv            resultados.csv


In [19]:
deportistaOlimpicoRDD = sc.textFile(PATH + 'deportista.csv') \
    .map(lambda l: l.split(','))
deportistaOlimpicoRDD2 = sc.textFile(PATH + 'deportista2.csv') \
    .map(lambda l: l.split(','))
deportistaOlimpicoRDD = deportistaOlimpicoRDD \
    .union(deportistaOlimpicoRDD2)

In [20]:
deportistaOlimpicoRDD.count()

135572

In [21]:
deportistaOlimpicoRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [22]:
deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]])) \
    .takeSample(False, 6, 25)

[('362', (['131505', 'Steven Woodburn', '1', '24', '185', '90'], 'FRA')),
 ('967', (['13626', 'Jill Brresen', '2', '22', '170', '57'], 'RSA')),
 ('482', (['44299', 'Gumundur Gumundsson', '1', '23', '174', '77'], 'ISL')),
 ('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('794', (['92442', 'Luis Paz Zoldan', '1', '19', '187', '82'], 'PER')),
 ('413', (['26822', 'Jared Mark Deacon', '1', '24', '185', '77'], 'GBR'))]

In [23]:
resultado = sc.textFile(PATH + 'resultados.csv') \
    .map(lambda l: l.split(','))

In [33]:
resultado = resultado.filter(lambda x: x[1] != 'NA')
resultado.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [59]:
'''Joining tables to show info'''
joinedInfo = deportistaOlimpicoRDD.map(lambda l: [l[0], l[1:]]) \
    .join(resultado.map(lambda x: [x[2], x[:2] + x[3:]])) \
    .map(lambda j: [j[1][0][-1], j]) \
    .join(equiposOlimpicosRDD.map(lambda i: [i[0], i])) \

In [60]:
valoresMedallas = {
    'Gold':7,
    'Silver':5,
    'Bronze':4
}

In [61]:
joinedInfo.takeSample(False, 2, 4)

[('399',
  (('4685',
    (['Katrin Apel', '2', '24', '172', '62', '399'],
     ['8518', 'Gold', '42', '311'])),
   ['399', 'Germany', 'GER'])),
 ('225',
  (('35261',
    (['scar Albeyro Figueroa Mosquera', '1', '21', '159', '62', '225'],
     ['69148', 'Gold', '51', '118'])),
   ['225', 'Colombia', 'COL']))]

In [62]:
paisesMedallas = joinedInfo.map(lambda x: [x[1][1][2], valoresMedallas.get(x[1][0][1][1][1])])

In [63]:
'''Counting points'''
from operator import add
conclusion = paisesMedallas.reduceByKey((add)).sortBy(lambda x: x[1],ascending=False)

In [64]:
conclusion.take(10)

[('USA', 32137),
 ('URS', 14834),
 ('GBR', 10925),
 ('GER', 10896),
 ('FRA', 9265),
 ('ITA', 8755),
 ('SWE', 8110),
 ('CAN', 7209),
 ('AUS', 6755),
 ('HUN', 6142)]

In [66]:
sc.stop()